# Make bilby samples from a list of galaxy hosts

This is a script to make bilby samples from a list of galaxy hosts. 

- The list of galaxy hosts is provided in a csv
- Galaxies are ingested, and the following quantities are propagated to the bilby sample
    - ra
    - dec
    - dL prior needs to be updated for the SkySim cosmology

In [1]:
import bilby as bb
import numpy as np
import gwcosmo as gwc
import pandas as pd
import GCR
import GCRCatalogs as GCRCat
import matplotlib.pyplot as plt
import os
from astropy.time import Time

ModuleNotFoundError: No module named 'bilby'

In [8]:
cat_name2 = "skysim5000_v1.2"
skysimCat = GCRCat.load_catalog(cat_name2) # Load the skysim catalog
hostDF = pd.read_csv("/global/u1/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/mockCBCCatalogs/mergers-w=Lum,n=1e7,FromSkySim50.csv") # Load the CBC catalog

In [9]:
prior = bb.gw.prior.BBHPriorDict(aligned_spin=True) # The bbh prior, spins misaligned
prior["luminosity_distance"] = bb.gw.prior.UniformSourceFrame(0,18000,cosmology=skysimCat.cosmology,name='luminosity_distance', latex_label='$d_L$', unit='Mpc', boundary=None) # Update the luminosity distance prior, based on 

10:04 bilby INFO    : Using aligned spin prior
10:04 bilby INFO    : No prior given, using default BBH priors in /pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/aligned_spins_bbh.prior.


In [10]:
prior

{'mass_1': Constraint(minimum=5, maximum=100, name='mass_1', latex_label='$m_1$', unit=None),
 'mass_2': Constraint(minimum=5, maximum=100, name='mass_2', latex_label='$m_2$', unit=None),
 'mass_ratio': bilby.gw.prior.UniformInComponentsMassRatio(minimum=0.125, maximum=1, name='mass_ratio', latex_label='$q$', unit=None, boundary=None, equal_mass=False),
 'chirp_mass': bilby.gw.prior.UniformInComponentsChirpMass(minimum=25, maximum=100, name='chirp_mass', latex_label='$\\mathcal{M}$', unit=None, boundary=None),
 'luminosity_distance': bilby.gw.prior.UniformSourceFrame(minimum=0.0, maximum=18000.0, cosmology=FlatLambdaCDM(name=None, H0=<Quantity 71. km / (Mpc s)>, Om0=0.2648, Tcmb0=<Quantity 0. K>, Neff=3.04, m_nu=None, Ob0=0.0448), name='luminosity_distance', latex_label='$d_L$', unit='Mpc', boundary=None),
 'dec': Cosine(minimum=-1.5707963267948966, maximum=1.5707963267948966, name='dec', latex_label='$\\mathrm{DEC}$', unit=None, boundary=None),
 'ra': Uniform(minimum=0, maximum=6.2831

In [11]:
bb.gw.prior.BBHPriorDict(aligned_spin=True)

12:33 bilby INFO    : Using aligned spin prior
12:33 bilby INFO    : No prior given, using default BBH priors in /pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/aligned_spins_bbh.prior.


{'mass_1': Constraint(minimum=5, maximum=100, name='mass_1', latex_label='$m_1$', unit=None),
 'mass_2': Constraint(minimum=5, maximum=100, name='mass_2', latex_label='$m_2$', unit=None),
 'mass_ratio': bilby.gw.prior.UniformInComponentsMassRatio(minimum=0.125, maximum=1, name='mass_ratio', latex_label='$q$', unit=None, boundary=None, equal_mass=False),
 'chirp_mass': bilby.gw.prior.UniformInComponentsChirpMass(minimum=25, maximum=100, name='chirp_mass', latex_label='$\\mathcal{M}$', unit=None, boundary=None),
 'luminosity_distance': bilby.gw.prior.UniformSourceFrame(minimum=100.0, maximum=5000.0, cosmology='Planck15', name='luminosity_distance', latex_label='$d_L$', unit='Mpc', boundary=None),
 'dec': Cosine(minimum=-1.5707963267948966, maximum=1.5707963267948966, name='dec', latex_label='$\\mathrm{DEC}$', unit=None, boundary=None),
 'ra': Uniform(minimum=0, maximum=6.283185307179586, name='ra', latex_label='$\\mathrm{RA}$', unit=None, boundary='periodic'),
 'theta_jn': Sine(minimum=0

In [12]:
bb.gw.prior.BBHPriorDict(aligned_spin=False)

12:33 bilby INFO    : No prior given, using default BBH priors in /pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/precessing_spins_bbh.prior.


{'mass_1': Constraint(minimum=5, maximum=100, name='mass_1', latex_label='$m_1$', unit=None),
 'mass_2': Constraint(minimum=5, maximum=100, name='mass_2', latex_label='$m_2$', unit=None),
 'mass_ratio': bilby.gw.prior.UniformInComponentsMassRatio(minimum=0.125, maximum=1, name='mass_ratio', latex_label='$q$', unit=None, boundary=None, equal_mass=False),
 'chirp_mass': bilby.gw.prior.UniformInComponentsChirpMass(minimum=25, maximum=100, name='chirp_mass', latex_label='$\\mathcal{M}$', unit=None, boundary=None),
 'luminosity_distance': bilby.gw.prior.UniformSourceFrame(minimum=100.0, maximum=5000.0, cosmology='Planck15', name='luminosity_distance', latex_label='$d_L$', unit='Mpc', boundary=None),
 'dec': Cosine(minimum=-1.5707963267948966, maximum=1.5707963267948966, name='dec', latex_label='$\\mathrm{DEC}$', unit=None, boundary=None),
 'ra': Uniform(minimum=0, maximum=6.283185307179586, name='ra', latex_label='$\\mathrm{RA}$', unit=None, boundary='periodic'),
 'theta_jn': Sine(minimum=0

In [6]:
skysimCat.cosmology.luminosity_distance(hostDF.loc[0]["redshiftHubble"])

<Quantity 23772.32428394 Mpc>

## Let's do the sampling this way
- For each entry in the CBC catalog
    - take a sample of the bilby prior, most parameters will be added
    - For ra/dec, use mra and mdec from the cbc catalog
    - For dL, just compute directly based on the cosmology of SkySim
- Add to the dataframe
- Save the dataframe to disk

In [6]:
keys = list(prior.sample())
keys.pop(3)
keys.pop(3)

'ra'

In [ ]:
injDict = {}
for k in keys:
    injDict[k] = []

cnt = 0
for ids,row in hostDF.iterrows():
    thisSample = prior.sample()
    for k in keys:
        if k!="luminosity_distance":
            injDict[k].append(thisSample[k])
    injDict["luminosity_distance"].append(float(str(skysimCat.cosmology.luminosity_distance(row["redshiftHubble"])).split(" ")[0]))
    if cnt % 100000 == 0:
        print("{}% finished".format(cnt//100000))
    cnt+=1

0% finished


In [ ]:
# for v in injDict.values():
#     print(np.shape(v))

In [ ]:
for k in injDict.keys():
    hostDF[k] = injDict[k]

In [ ]:
float(str(skysimCat.cosmology.luminosity_distance(row["redshiftHubble"])).split(" ")[0])

## Save this

In [ ]:
saveColumns = hostDF.columns.values[1:] # This is because of the extra column - check this always!!!

In [ ]:
dataDir = "/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data"
hostDF.to_csv(os.path.join(dataDir,"mergers-w=Lum,n=1e7,FromSkySim50_withBilby.csv"),columns=saveColumns,index=False)

## OOPS! Add a column to denote if the CBC has been sampled before

In [ ]:
hostDF = pd.read_csv(os.path.join(dataDir,"mergers-w=Lum,n=1e7,FromSkySim50_withBilby.csv"))

In [ ]:
hostDF["sampled"] = False

In [ ]:
hostDF.to_csv(os.path.join(dataDir,"mergers-w=Lum,n=1e7,FromSkySim50_withBilby.csv"),columns=saveColumns,index=False)